# Workflow to Validate NISAR L2 Coseismic Displacement Requirement

**Original code authored by:** David Bekaert, Heresh Fattahi, Eric Fielding, and Zhang Yunjun 

Extensive modifications by Adrian Borsa and Amy Whetter 2022

<div class="alert alert-warning">
Both the initial setup (<b>Prep A</b> section) and download of the data (<b>Prep B</b> section) should be run at the start of the notebook. And all subsequent sections NEED to be run in order.
</div>

<hr/>

## Define CalVal Site 

In [ ]:
# Choose a site and track direction
site='MojaveD173' 

# What dataset are you processing?
dataset = 'ARIA_S1' # For Sentinel-1 testing with aria-tools

# The date and version of this Cal/Val run
today = '20240429'
version = '1'

# Define your directory structure - you won't need to change this line
start_directory = '/scratch/nisar-st-calval-solidearth' 

# The file where you keep your customized list of sites.
custom_sites = '/home/jovyan/my_sites.txt'

# Please enter a name or username that will determine where your outputs are stored
import os
if os.path.exists('/home/jovyan/me.txt'): # if OpenTopo API key already installed
    with open('/home/jovyan/me.txt') as m:
        you = m.readline().strip()
    print('You are', you)
    print('Using this as the name of the directory where your outputs will be stored.')
    print('Directory structure: start_directory / dataset/ requirement / site / you / today / version ')
    print('For example: /scratch/nisar-st-calval-solidearth/ARIA_S1/Coseismic/MojaveD173/aborsa/20240424/v1/')
else:
    print('We need a name or username (determines where your outputs will be stored)')
    print('Directory structure: start_directory / dataset/ requirement / site / you / today / version ')
    print('For example: /scratch/nisar-st-calval-solidearth/ARIA_S1/Coseismic/MojaveD173/aborsa/20240424/v1/')
    you = input('Please type your name:')
    with open ('/home/jovyan/me.txt', 'w') as m: 
        m.write(you)
    

## Table of Contents:
<a id='example_TOC'></a>

<hr/>

[**Environment Setup**](#prep_a)

[**1. Generation of Time Series from Interferograms**](#gen_ts)
- [1.1. Validate/Modify Interferogram Network](#validate_network)
- [1.2. Generate Quality Control Mask](#generate_mask)
- [1.3. Reference Interferograms To Common Lat/Lon](#common_latlon)
- [1.4. Invert for SBAS Line-of-Sight Timeseries](#invert_SBAS)

[**2. Optional Corrections**](#opt_correction)
- [2.1. Solid Earth Tides Correction](#solid_earth)
- [2.2. Tropospheric Delay Correction](#tropo_corr)
- [2.3. Phase Deramping ](#phase_deramp)
- [2.4. Topographic Residual Correction ](#topo_corr) 

[**3. Decomposition of InSAR and GNSS Time Series into Basis Functions**](#decomp_ts)
- [3.1. Estimate InSAR LOS Velocities](#insar_vel1)
- [3.2. Estimate InSAR Coseismic Displacement](#co_step1)
- [3.3. Find Collocated GNSS Stations](#co_gps)  
- [3.4. Get GNSS Position Time Series](#gps_ts) 
- [3.5. Make GNSS LOS Velocities](#gps_los)
- [3.6. Re-reference GNSS and InSAR LOS Coseismic Step](#gps_insar)

[**4. NISAR Validation Approach 1: GNSS-InSAR Direct Comparison**](#nisar_validation)
- [4.1. Make Displacement Residuals at GNSS Locations](#make_vel)
- [4.2. Make Double-differenced Displacement Residuals](#make_velres1)
- [4.3. Amplitude vs. Distance of Double-differences (not quite a structure function)](#amp_vs_dist)
- [4.4. GNSS-InSAR Residuals Analysis with Step Function](#nisar_anal)
- [4.5. Make Double-differenced Displacement Residuals Step](#make_velres2)
- [4.6. Amplitude vs. Distance of Double-differences Step (not quite a structure function)](#ampvsdist2)
- [4.7. Coseismic Requirement Validation: Method 1](#coseismic_valid_method1)

[**5. NISAR Validation Approach 2: InSAR-only Structure Function**](#nisar_validation2)
- [5.1. Read Array and Mask Pixels with no Data](#array_mask)
- [5.2. Randomly Sample Pixels and Pair Them Up with Option to Remove Trend](#remove_trend)
- [5.3. Amplitude vs. Distance of Relative Measurements (pair differences)](#M2ampvsdist2)
- [5.4. Bin Sample Pairs by Distance Bin and Calculate Statistics](#M2RelMeasTable)

[**Appendix: Supplementary Comparisons and Plots**](#appendix)
- [A.1. Compare Raw Velocities](#compare_raw)
- [A.2. Plot Velocity Residuals](#plot_vel)
- [A.3. Plot Double-differenced Residuals](#plot_velres)
- [A.4. GPS Position Plot](#appendix_gps)

<hr/>

# Environment setup

Find your workspace and the MintPy-readable stack you created previously with one of the data prep notebooks.

In [ ]:
#Load Packages
import glob
import os
import subprocess
from datetime import datetime as dt
from pathlib import Path
import json
import math
import pickle

import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from mintpy.cli import view, plot_network
from mintpy.objects import gnss, timeseries
from mintpy.smallbaselineApp import TimeSeriesAnalysis
from mintpy.utils import ptime, readfile, time_func, utils as ut
from scipy import signal

from solid_utils.sampling import load_geo, samp_pair, profile_samples, haversine_distance

#Set Global Plot Parameters
plt.rcParams.update({'font.size': 12})

################# Set Directories ##########################################
requirement='Coseismic'
work_dir = os.path.join(start_directory,dataset,requirement,site,you,today,'v'+version)
print("Work directory:", work_dir)

gunw_dir = os.path.join(work_dir,'products')
print("   GUNW    dir:", gunw_dir) 

mintpy_dir = os.path.join(work_dir,'MintPy')
print("   MintPy  dir:", mintpy_dir)
### Change to MintPy workdir
if not os.path.exists(mintpy_dir):
    print()
    print('ERROR: Stop! Your MintPy processing directory does not exist for this requirement, site, version, or date of your ATBD run.')
    print('You may need to run the prep notebook first!')
    print()
else:
    os.chdir(mintpy_dir)

vel_file = os.path.join(mintpy_dir, 'velocity.h5')
msk_file = os.path.join(mintpy_dir, 'maskConnComp.h5')  # maskTempCoh.h5 maskConnComp.h5

with open(custom_sites,'r') as fid:
    sitedata = json.load(fid)

sitedata['sites'][site]

# 1. Generation of Time Series from Interferograms

<a id='validate_network'></a>
## 1.1. Validate/Modify Interferogram Network

Add additional parameters to config_file in order to remove selected interferograms, change minimum coherence, etc.

In [ ]:
config_file = os.path.join(mintpy_dir,site + '.cfg')
command = 'smallbaselineApp.py ' + str(config_file) + ' --dostep modify_network'
process = subprocess.run(command, shell=True)

plot_network.main(['inputs/ifgramStack.h5'])

<a id='generate_mask'></a>
## 2.4. Generate Quality Control Mask


Mask files can be can be used to mask pixels in the time-series processing. Below we generate an initial mask file `maskConnComp.h5` based on the connected components for all the interferograms, which is a metric for unwrapping quality. After time-series analysis is complete, we will calculate a mask from the temporal coherence or variation of phase or displacement with time to make `maskTempCoh.h5`.

In [ ]:
command='generate_mask.py inputs/ifgramStack.h5  --nonzero  -o maskConnComp.h5  --update'
process = subprocess.run(command, shell=True)
view.main(['maskConnComp.h5', 'mask'])
msk_file = os.path.join(mintpy_dir, 'maskTempCoh.h5')

<a id='common_latlon'></a>
## 1.2. Reference Interferograms To Common Lat/Lon


In [ ]:
command = 'smallbaselineApp.py ' + str(config_file) + ' --dostep reference_point'
process = subprocess.run(command, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True, shell=True)
os.system('info.py inputs/ifgramStack.h5 | egrep "REF_"');

<a id='invert_SBAS'></a>
## 1.3. Invert for SBAS Line-of-Sight Timeseries


In [ ]:
command = 'smallbaselineApp.py ' + str(config_file) + ' --dostep invert_network'
process = subprocess.run(command, shell=True)

<a id='opt_correction'></a>
# 2. Optional Corrections

Phase distortions related to solid earth and ocean tidal effects as well as those due to temporal variations in the vertical stratification of the atmosphere can be mitigated using the approaches described below. At this point, it is expected that these corrections will not be needed to validate the mission requirements, but they may be used to produce the highest quality data products. Typically, these are applied to the estimated time series product rather than to the individual interferograms, since they are a function of the time of each radar acquisition.

<a id='solid_earth'></a>
## 2.1. Solid Earth Tides Correction

[MintPy provides functionality for this correction.]

<a id='tropo_corr'></a>
## 2.2. Tropospheric Delay Correction

Optional atmospheric correction utilizes the PyAPS (Jolivet et al., 2011, Jolivet and Agram, 2012) module within GIAnT (or eventually a merged replacement for GIAnT and MintPy). PyAPS is well documented, maintained and can be freely downloaded. PyAPS is included in GIAnT distribution). PyAPS currently includes support for ECMWF’s ERA-Interim, NOAA’s NARR and NASA’s MERRA weather models. A final selection of atmospheric models to be used for operational NISAR processing will be done during Phase C.

[T]ropospheric delay maps are produced from atmospheric data provided by Global Atmospheric Models. This method aims to correct differential atmospheric delay correlated with the topography in interferometric phase measurements. Global Atmospheric Models (hereafter GAMs)... provide estimates of the air temperature, the atmospheric pressure and the humidity as a function of elevation on a coarse resolution latitude/longitude grid. In PyAPS, we use this 3D distribution of atmospheric variables to determine the atmospheric phase delay on each pixel of each interferogram.

The absolute atmospheric delay is computed at each SAR acquisition date. For a pixel a_i at an elevation z at acquisition date i, the four surrounding grid points are selected and the delays for their respective elevations are computed. The resulting delay at the pixel a_i is then the bilinear interpolation between the delays at the four grid points. Finally, we combine the absolute delay maps of the InSAR partner images to produce the differential delay maps used to correct the interferograms.

[MintPy provides functionality for this correction.]

In [ ]:
do_tropo_correction = False
########################################################################
'''
REFERENCE : https://github.com/insarlab/pyaps#2-account-setup-for-era5
Read Section 2 for ERA5 [link above] to create an account on the CDS website.
'''

if do_tropo_correction:
    if not Use_Staged_Data and not os.path.exists(Path.home()/'.cdsapirc'):
        print('NEEDED to download ERA5, link: https://cds.climate.copernicus.eu/user/register')
        UID = input('Please type your CDS_UID:')
        CDS_API = input('Please type your CDS_API:')
        
        cds_tmp = '''url: https://cds.climate.copernicus.eu/api/v2
        key: {UID}:{CDS_API}'''.format(UID=UID, CDS_API=CDS_API)
        os.system('echo "{cds_tmp}" > ~/.cdsapirc; chmod 600 ~/.cdsapirc'.format(cds_tmp = str(cds_tmp)))
    
    command = 'smallbaselineApp.py ' + str(config_file) + ' --dostep correct_troposphere'
    process = subprocess.run(command, shell=True)
    
    view.main(['inputs/ERA5.h5'])
    timeseries_filename = 'timeseries_ERA5.h5'
else:
    timeseries_filename = 'timeseries.h5'

<a id='phase_deramp'></a>
## 2.3. Phase Deramping


In [ ]:
command = 'smallbaselineApp.py ' + str(config_file) + ' --dostep deramp'
process = subprocess.run(command, shell=True)

<a id='topo_corr'></a>
## 2.4. Topographic Residual Correction

[MintPy provides functionality for this correction.]

In [ ]:
command = 'smallbaselineApp.py ' + str(config_file) + ' --dostep correct_topography'
process = subprocess.run(command, shell=True)

<a id='decomp_ts'></a>
# 3. Decomposition of InSAR and GNSS Time Series into Basis Functions


<a id='insar_vel1'></a>
## 3.1. Estimate InSAR LOS Velocities

Given a time series of InSAR LOS displacements, the observations for a given pixel, $U(t)$, can be parameterized as:

$$ U(t) = a \;+\; vt \;+\; c_1 cos (\omega_1t - \phi_{1,}) \;+\; c_2 cos (\omega_2t - \phi_2) \;+\; \sum_{j=1}^{N_{eq}} \left( h_j+f_j F_j (t-t_j) \right)H(t - t_j) \;+\; \frac{B_\perp (t)}{R sin \theta}\delta z \;+\; residual$$ 

which includes a constant offset $(a)$, velocity $(v)$, and amplitudes $(c_j)$ and phases $(\phi_j)$ of annual $(\omega_1)$ and semiannual $(\omega_2)$ sinusoidal terms.  Where needed we can include additional complexity, such as coseismic and postseismic processes parameterized by Heaviside (step) functions $H$ and postseismic functions $F$ (the latter typically exponential and/or logarithmic).   $B_\perp(t)$, $R$, $\theta$, and $\delta z$ are, respectively, the perpendicular component of the interferometric baseline relative to the first date, slant range distance, incidence angle and topography error correction for the given pixel. 

Thus, given either an ensemble of interferograms or the output of SBAS (displacement vs. time), we can write the LSQ problem as 

$$ \textbf{G}\textbf{m} = \textbf{d}$$

where $\textbf{G}$ is the design matrix (constructed out of the different functional terms in Equation 2 evaluated either at the SAR image dates for SBAS output, or between the dates spanned by each pair for interferograms), $\textbf{m}$ is the vector of model parameters (the coefficients in Equation 2) and $\textbf{d}$ is the vector of observations.  For GPS time series, $\textbf{G}, \textbf{d}, \textbf{m}$ are constructed using values evaluated at single epochs corresponding to the GPS solution times, as for SBAS InSAR input. 

With this formulation, we can obtain InSAR velocity estimates and their formal uncertainties (including in areas where the expected answer is zero). 

The default InSAR velocity fit in MintPy is to estimate a mean linear velocity $(v)$ in in the equation, which we do below. This is the same linear velocity used in the Secular notebook.

In [ ]:
print(mintpy_dir)
#command = 'smallbaselineApp.py ' + str(config_file) + ' --dostep velocity'
command = 'timeseries2velocity.py ' + timeseries_filename 
process = subprocess.run(command, shell=True)
vel_file = os.path.join(mintpy_dir, 'velocity.h5')
vel = readfile.read(vel_file, datasetName = 'velocity')[0] * 100.  #read and convert velocities from m to cm

# optionally set masked pixels to NaN
#msk = readfile.read(msk_file)[0]
#vel[msk == 0] = np.nan

Now we plot the mean linear velocity fit. The MintPy `view` module automatically reads the temporal coherence mask `maskTempCoh.h5` and applies that to mask out pixels with unreliable velocities (white). Because there is a large coseismic displacement at the time of the earthquake but we are only estimating a single linear velocity, it absorbs the coseismic signal in this plot.

In [ ]:
scp_args = 'velocity.h5 velocity -v -25 25 --ref-yx 200 950 --colormap RdBu --figtitle LOS_Velocity' # --plot-setting ' + plot_config_file
view.main(scp_args.split())

<div class="alert alert-info">
<b>Note :</b> 
Negative values indicates that target is moving away from the radar (i.e., Subsidence in case of vertical deformation).
Positive values indicates that target is moving towards the radar (i.e., uplift in case of vertical deformation). 
</div>

<a id='co_step1'></a>
## 3.2. Estimate InSAR Coseismic Displacement

We can use the same time series to estimate the coseismic displacement as a Heaviside $H$ or step function at the time of an earthquake or a number $N_{eq}$ of earthquakes. In the above equation this is the $$\sum_{j=1}^{N_{eq}} \left( h_j+f_j F_j (t-t_j) \right)H(t - t_j) \;$$ set of terms. For simplicity, we consider only one earthquake and we assume the postseismic displacement functions $F$ are small compared to the coseismic displacements, so we only need to solve for the coefficient $h$ of each interferogram pixel.

We call the MintPy `timeseries2velocity.py` program again and specify the time of the earthquake $t_j$. The fit will also include the linear velocity rate separated from the step function amplitude. Both estimated coefficients have their associated uncertainties.

In [ ]:
command = 'timeseries2velocity.py ' + timeseries_filename + ' --step ' + sitedata['sites'][site]['earthquakeDate']
process = subprocess.run(command, shell=True)

EQdataset = 'step' + sitedata['sites'][site]['earthquakeDate']
EQstep = readfile.read(vel_file, datasetName = EQdataset)[0] * 100.  #read and convert coseismic step from m to cm

Now we can view the step function amplitude and the new linear velocity estimate.

In [ ]:
scp_args = 'velocity.h5 step' + sitedata['sites'][site]['earthquakeDate'] + ' -v -40 40 --ref-yx 200 950 --colormap RdBu --figtitle LOS_Coseismic' # --plot-setting ' + plot_config_file
view.main(scp_args.split())
scp_args = 'velocity.h5 velocity -v -25 25 --ref-yx 200 950 --colormap RdBu --figtitle LOS_Velocity' # --plot-setting ' + plot_config_file
view.main(scp_args.split())

<a id='co_gps'></a>
## 3.3. Find Collocated GNSS Stations

The project will have access to L2 position data for continuous GNSS stations in third-party networks such NSF’s Plate Boundary Observatory, the HVO network for Hawaii, GEONET-Japan, and GEONET-New Zealand, located in target regions for NISAR solid earth calval. Station data will be post-processed by one or more analysis centers, will be freely available, and will have latencies of several days to weeks, as is the case with positions currently produced by the NSF’s GAGE Facility and separately by the University of Nevada Reno. Networks will contain one or more areas of high-density station coverage (2~20 km nominal station spacing over 100 x 100 km or more) to support validation of L2 NISAR requirements at a wide range of length scales.

In [ ]:
# get analysis metadata from InSAR velocity file
atr = readfile.read_attribute(vel_file)
length, width = int(atr['LENGTH']), int(atr['WIDTH'])
lat_step = float(atr['Y_STEP'])
lon_step = float(atr['X_STEP'])
N = float(atr['Y_FIRST'])
W = float(atr['X_FIRST'])
S = N + lat_step * length
E = W + lon_step * width
start_date = atr.get('START_DATE', None)
end_date = atr.get('END_DATE', None)
start_date_gnss = dt.strptime(start_date, "%Y%m%d")
end_date_gnss = dt.strptime(end_date, "%Y%m%d")
#inc_angle = int(float(atr.get('incidenceAngle', None)))
#az_angle = int(float(atr.get('azimuthAngle', None))) 
inc_angle = 32.5
az_angle = 0.0

#Set GNSS Parameters
gnss_completeness_threshold = 0.9     #0.9  #percent of data timespan with valid GNSS epochs
gnss_residual_stdev_threshold = 20.   #0.03  #0.03  #max threshold standard deviation of residuals to linear GNSS fit

# search for collocated GNSS stations
site_names, site_lats, site_lons = gnss.search_gnss(SNWE=(S,N,W,E), 
                                                  start_date=start_date_gnss.strftime('%Y%m%d'),
                                                  end_date=end_date_gnss.strftime('%Y%m%d'))
site_names = [str(stn) for stn in site_names]
print("Initial list of {} stations used in analysis:".format(len(site_names)))
print(site_names)

<a id='gps_ts'></a>
## 3.4. Get GNSS Position Time Series


In [ ]:
# get daily position solutions for GNSS stations
use_stn = []  #stations to keep
bad_stn = []  #stations to toss
use_lats = [] 
use_lons = []
counter = 0

for stn in site_names:
    gnss_obj = gnss.get_gnss_class('UNR')(site = stn, data_dir = os.path.join(mintpy_dir,'GNSS'))
    gnss_obj.open(print_msg='False')
    
    # count number of dates in time range
    dates = gnss_obj.dates
    range_days = (end_date_gnss - start_date_gnss).days
    gnss_count = np.histogram(dates, bins=[start_date_gnss,end_date_gnss])
    gnss_count = int(gnss_count[0])
    
    # for this quick screening check of data quality, we use the constant incidence and azimuth angles 
    # get standard deviation of residuals to linear fit
    disp_los = ut.enu2los(gnss_obj.dis_e, gnss_obj.dis_n, gnss_obj.dis_u, inc_angle, az_angle)
    #disp_los = ut.enu2los(gps_obj.dis_e, gps_obj.dis_n, gps_obj.dis_u, inc_angle, head_angle, az_angle)
    disp_detrended = signal.detrend(disp_los)
    stn_stdv = np.std(disp_detrended)
   
    # select GNSS stations based on data completeness and scatter of residuals
    disp_detrended = signal.detrend(disp_los)
    if range_days*gps_completeness_threshold <= gnss_count:
        
        if stn_stdv > gps_residual_stdev_threshold:
            bad_stn.append(stn)
        else:
            use_stn.append(stn)
            use_lats.append(site_lats[counter])
            use_lons.append(site_lons[counter])
    else:
        bad_stn.append(stn)
    counter+=1

site_names = use_stn
site_lats = use_lats
site_lons = use_lons

# [optional] manually remove additional stations
gnns_to_remove=['CAL8', 'P094', 'COSO', 'CAC2']

for i, gnss_site in enumerate(gnns_to_remove):
    if gnss_site in site_names:
        site_names.remove(gnss_site)
    if gnss_site not in bad_stn:
        bad_stn.append(gnss_site)
    
print("Final list of {} stations used in analysis:".format(len(site_names)))
print(site_names)
print("List of {} stations removed from analysis".format(len(bad_stn)))
print(bad_stn)

<a id='gps_los'></a>
## 3.5. Make GNSS LOS Velocities

As a first approximation to estimate coseismic displacements, we take one day before and one (or three) day after earthquake.

In [ ]:
EQdate_int = int(sitedata['sites'][site]['earthquakeDate'])
EQpre_date = str(EQdate_int - 1)  # should convert to real date object first
EQpost_date = str(EQdate_int + 3)

gnss_comp = 'enu2los'
meta = readfile.read_attribute('velocity.h5')
SNWE = ut.four_corners(meta)
vel = gnss.get_los_obs(meta, 'velocity',     site_names, start_date='20150101', end_date='20190619')
displ = gnss.get_los_obs(meta, 
                            'displacement', 
                            site_names, 
                            start_date=EQpre_date, 
                            end_date=EQpost_date,
                            gnss_comp=gnss_comp, 
                            redo=True) * 100.
print('\n site   disp-los [cm/yr]')
print(np.array([site_names,displ]).T)

In [ ]:
# 1. download & read GNSS displacement time series
site_id = 'RAMT'
gnss_obj = gnss.get_gnss_class('UNR')(site = stn, data_dir = os.path.join(mintpy_dir,'GNSS'))
gnss_obj.open()

# 2. fit time function
model = {
    'polynomial' : 1,
    'stepDate'       : [sitedata['sites'][site]['earthquakeDate']]
}
G, m, e2 = time_func.estimate_time_func(model, gnss_obj.date_list, gnss_obj.dis_n)

print('fit parameters (constant, linear velocity, step)',m)

# 3. reconstruct time series from estimated time function parameters
date_list_fit = ptime.get_date_range(gnss_obj.date_list[0], gnss_obj.date_list[-1])
dates_fit = ptime.date_list2vector(date_list_fit)[0]
G_fit = time_func.get_design_matrix4time_func(date_list_fit, model)
dis_ts_fit = np.matmul(G_fit, m)

# plot
fig, ax = plt.subplots(figsize=[12, 4])
ax.plot(gnss_obj.dates, gnss_obj.dis_n, '.', ms=4)
ax.plot(dates_fit, dis_ts_fit, lw=4, alpha=0.8)
ax.set_ylabel('North displacement [m]')
ax.set_title(f'Site {site_id} at [{gnss_obj.site_lon:.6f}, {gnss_obj.site_lat:.6f}]')
plt.show()

In [ ]:
# get InSAR geometry file
geom_file = ut.get_geometry_file(['incidenceAngle','azimuthAngle'], work_dir=mintpy_dir, coord='geo')

# 1. download & read GNSS displacement time series
site_id = 'RAMT'
gnss_obj = gnss.get_gnss_class('UNR')(site = stn, data_dir = os.path.join(mintpy_dir,'GNSS'))
gnss_obj.open()

# 2. fit time function
model = {
    'polynomial' : 1,
    'stepDate'       : [sitedata['sites'][site]['earthquakeDate']]
}

# project the GNSS three components to InSAR LOS
dis_los = ut.enu2los(gnss_obj.dis_e,gnss_obj.dis_n,gnss_obj.dis_u,inc_angle,az_angle)

# fit whole time series for this station
# date range for this station
statStart = gnss_obj.date_list[0]
statEnd = gnss_obj.date_list[-1]


#dates, dis_los, std, site_lalo, ref_site_lalo = gps_obj.read_gps_los_displacement(geom_file, 
#                                                                                  start_date=statStart, 
#                                                                                  end_date=statEnd, 
#                                                                                  ref_site=None,
#                                                                                  gps_comp=gps_comp, 
#                                                                                  print_msg=False)

G, m, e2 = time_func.estimate_time_func(model, gnss_obj.date_list, dis_los)

print('fit parameters (constant, linear velocity, step)',m)

# 3. reconstruct time series from estimated time function parameters
date_list_fit = ptime.get_date_range(statStart, statEnd)
dates_fit = ptime.date_list2vector(date_list_fit)[0]
G_fit = time_func.get_design_matrix4time_func(date_list_fit, model)
dis_ts_fit = np.matmul(G_fit, m)

# plot
fig, ax = plt.subplots(figsize=[12, 4])
ax.plot(gnss_obj.dates, dis_los, '.', ms=4)
ax.plot(dates_fit, dis_ts_fit, lw=4, alpha=0.8)
ax.set_ylabel('LOS displacement [m]')
ax.set_title(f'Site {site_id} at [{gnss_obj.site_lon:.6f}, {gnss_obj.site_lat:.6f}]')
plt.show()

Now do the LOS fits for all good GPS stations

In [ ]:
gnss_los_coseis_cm = []
plotGNSSfit = False

# need to update these to keep consistent
site_lats = []
site_lons = []

twoQuake = False
# fit time function
if twoQuake:
    modelParam = {
        'polynomial' : 1,
        'stepDate'       : [sitedata['sites'][site]['earthquakeDate'], sitedata['sites'][site]['earthquakeDate2']],
    }
else:
    modelParam = {
        'polynomial' : 1,
        'stepDate'       : [sitedata['sites'][site]['earthquakeDate']],
    }

for stn in site_names:
    gnss_obj = gnss.get_gnss_class('UNR')(site = stn, data_dir = os.path.join(mintpy_dir,'GNSS'))
    gnss_obj.open()
    
    site_lats.append(gnss_obj.site_lat) # save site location in list
    site_lons.append(gnss_obj.site_lon)
    
    # fit whole time series for this station
    # date range for this station
    statStart = gnss_obj.date_list[0]
    statEnd = gnss_obj.date_list[-1]

    dates, dis_los, std, site_lalo, ref_site_lalo = gnss_obj.get_los_displacement(geom_file, 
                                                                                      start_date=statStart, 
                                                                                      end_date=statEnd, 
                                                                                      ref_site=None,
                                                                                      gnss_comp=gnss_comp, 
                                                                                      print_msg=False)

    
    G, m, e2 = time_func.estimate_time_func(model, gnss_obj.date_list, dis_los)

    print('station',stn,'fit parameters (constant, linear velocity, step)',m)
    gnss_los_coseis_cm.append(m[2]*100.)  # save step function and convert to cm for convenience
    
    if (plotGNSSfit):
        # 3. reconstruct time series from estimated time function parameters
        date_list_fit = ptime.get_date_range(gnss_obj.date_list[0], gnss_obj.date_list[-1])
        dates_fit = ptime.date_list2vector(date_list_fit)[0]
        G_fit = time_func.get_design_matrix4time_func(date_list_fit, model)
        dis_ts_fit = np.matmul(G_fit, m)
        
        dates = gnss_obj.date_list
        index_start = dates.index(gnss_obj.date_list[0])
        index_end = dates.index(gnss_obj.date_list[-1])
        print ('index_start',index_start,'end',index_end)
        print('len dates',len(dates),'dis_los',len(dis_los),'dates_fit',len(dates_fit))
        cut_dates = dates[index_start:index_end]
        cut_dates_vec = ptime.date_list2vector(cut_dates)[0]
        cut_dis_los = dis_los[index_start:index_end]

        # plot
        fig, ax = plt.subplots(figsize=[12, 4])
        ax.plot(cut_dates_vec, cut_dis_los, '.', ms=4)
        ax.plot(dates_fit, dis_ts_fit, lw=4, alpha=0.8)
        ax.set_ylabel('LOS displacement [m]')
        ax.set_title(f'Site {stn} at [{gnss_obj.site_lon:.6f}, {gnss_obj.site_lat:.6f}]')
        plt.show()
        
    
print(gnss_los_coseis_cm)

# compare the simple difference displacements to the time-series step fits
dispDiff = displ - gnss_los_coseis_cm
print('difference in displacement estimate (cm)\n', dispDiff)
print('difference in coseismic displacement estimate (%)\n',(displ + gnss_los_coseis_cm) / 2)

<a id='gps_insar'></a>
## 3.6. Re-reference GNSS and InSAR LOS Coseismic Step


In [ ]:
# reference GNSS stations to GNSS reference site
ref_site_ind = site_names.index(sitedata['sites'][site]['gps_ref_site_name'])
displRelRef = displ - displ[ref_site_ind]

# reference InSAR to GNSS reference site
ref_site_lat = float(site_lats[ref_site_ind])
ref_site_lon = float(site_lons[ref_site_ind])
ref_y, ref_x = ut.coordinate(atr).geo2radar(ref_site_lat, ref_site_lon)[:2]
print ('new reference pixel for InSAR y,x:',ref_y, ref_x,'at lat, lon',ref_site_lat,ref_site_lon)
EQstep = EQstep - EQstep[ref_y, ref_x]

In [ ]:
# plot GNSS stations on InSAR coseismic field
vmin, vmax = -40, 40
cmap = plt.get_cmap('RdBu')

fig, ax = plt.subplots(figsize=[18, 5.5])
img1 = ax.imshow(EQstep, cmap=cmap, vmin=vmin, vmax=vmax, interpolation='nearest', extent=(W, E, S, N))
cbar1 = fig.colorbar(img1, ax=ax)
cbar1.set_label('LOS EQ displacement')

for lat, lon, obs in zip(site_lats, site_lons, displRelRef):
    color = cmap((obs - vmin)/(vmax - vmin))
    ax.scatter(lon, lat, color=color, s=8**2, edgecolors='k')
for i, label in enumerate(site_names):
     plt.annotate(label, (site_lons[i], site_lats[i]), color='black')

out_fig = os.path.abspath('coseismic_insar_vs_gnss.png')
fig.savefig(out_fig, bbox_inches='tight', transparent=True, dpi=300)

In [ ]:
# same re-reference for the time-series GNSS fit
ref_site_ind = site_names.index(sitedata['sites'][site]['gps_ref_site_name'])
displFitRelRef = gnss_los_coseis_cm - gnss_los_coseis_cm[ref_site_ind]

# plot GNSS stations on InSAR coseismic field
vmin, vmax = -40, 40
cmap = plt.get_cmap('RdBu')

fig, ax = plt.subplots(figsize=[18, 5.5])
img1 = ax.imshow(EQstep, cmap=cmap, vmin=vmin, vmax=vmax, interpolation='nearest', extent=(W, E, S, N))
cbar1 = fig.colorbar(img1, ax=ax)
cbar1.set_label('LOS EQ displacement')

for lat, lon, obs in zip(site_lats, site_lons, displFitRelRef):
    color = cmap((obs - vmin)/(vmax - vmin))
    ax.scatter(lon, lat, color=color, s=8**2, edgecolors='k')
for i, label in enumerate(site_names):
     plt.annotate(label, (site_lons[i], site_lats[i]), color='black')

out_fig = os.path.abspath('coseismic_insar_vs_gnss_fit.png')
fig.savefig(out_fig, bbox_inches='tight', transparent=True, dpi=300)

<a id='nisar_validation'></a>
# 4. NISAR Validation Approach 1: GNSS-InSAR Direct Comparison 


<a id='make_vel'></a>
## 4.1. Make Displacement Residuals at GNSS Locations


First compare the coseismic displacements estimated with the simple subtraction of pre-quake and post-quake dates with the GNSS or GPS displacements.

In [ ]:
#Set Parameters
pixel_radius = 5   #number of InSAR pixels to average for comparison with GNSS

#Create dictionary with the stations as the key and all their info as an array 
stn_dict = {}  

#Loop over GNSS station locations
for i in range(0,len(site_names)): 
    #print(site_names[i])
    
    # convert GNSS station lat/lon information to InSAR x/y grid
    stn_lat = site_lats[i]
    stn_lon = site_lons[i]
    x_value = round((stn_lon - W)/lon_step)
    y_value = round((stn_lat - N)/lat_step)
    
    # get velocities and residuals
    disp_GNSS = displ[i]
    #Caution: If you expand the radius parameter farther than the bounding grid it will break. 
    #To fix, remove the station in section 4 when the site_names list is filtered
    disp_px_rad = EQstep[y_value-pixel_radius:y_value+1+pixel_radius, 
                     x_value-pixel_radius:x_value+1+pixel_radius]
    disp_InSAR = np.median(disp_px_rad)
    residual = disp_GNSS - disp_InSAR

    # populate data structure
    values = [x_value, y_value, disp_InSAR, disp_GNSS, residual, stn_lat, stn_lon]
    stn = site_names[i]
    stn_dict[stn] = values
    
# extract data from structure
res_list1 = []
insar_disp1 = []
gnss_disp1 = []
lat_list1 = []
lon_list1 = []
for i in range(len(site_names)): 
    stn = site_names[i]
    insar_disp1.append(stn_dict[stn][2])
    gnss_disp1.append(stn_dict[stn][3])
    res_list1.append(stn_dict[stn][4])
    lat_list1.append(stn_dict[stn][5])
    lon_list1.append(stn_dict[stn][6])
num_stn = len(site_names)
print('Finish creating InSAR residuals at GNSS sites')

<a id='make_velres1'></a>
## 4.2. Make Double-differenced Displacement Residuals

In [ ]:
diff_res_list1 = []
stn_dist_list1 = []
dict_keys = list(stn_dict.keys())

# remove reference stn
site_names_analysis = list(site_names)
#site_names_analysis.remove(gps_ref_site_name)

# loop over stations
for i in range(len(site_names_analysis)-1):
    stn1 = dict_keys[i]
    print(site_names_analysis[i])
    for l in range(i + 1, len(dict_keys)):
        stn2 = dict_keys[l]
        # calculate between-station velocity residual
        # stn_dict values = [x_value, y_value, vel_InSAR, vel_GPS, residual, stn_lat, stn_lon]
        # index 3 = gps vel
        gnss_vel_diff = stn_dict[stn1][3] - stn_dict[stn2][3]
        # index 2 = insar vel 
        insar_vel_diff = stn_dict[stn1][2]-stn_dict[stn2][2]
        # calculate double-difference
        diff_res = gnss_vel_diff - insar_vel_diff
        diff_res_list1.append(diff_res)
        # get distance between selected station
        # index 5 is lat, 6 is lon
        dlat = (stn_dict[stn1][5]-stn_dict[stn2][5])
        dlon = (stn_dict[stn1][6]-stn_dict[stn2][6])*np.sin(stn_dict[stn1][5])
        #convert degrees to km
        stn_dist = math.sqrt(dlat**2 + dlon**2)*111
        stn_dist_list1.append(stn_dist)

<a id='amp_vs_dist'></a>
## 4.3. Amplitude vs. Distance of Double-differences (not quite a structure function)

In [ ]:
dist_th = np.linspace(0.1,50,100)  # distances for evaluation
acpt_error = 4*(1+np.sqrt(dist_th))  # coseismic threshold in mm
acpt_error_cm = acpt_error/10.
abs_ddiff_disp = [abs(i) for i in diff_res_list1]

fig, ax = plt.subplots(figsize=(11,7))
plt.scatter(stn_dist_list1,abs_ddiff_disp,label='D_gnss - D_InSAR for station pair')
plt.plot(dist_th, acpt_error_cm, 'r',label='Coseismic requirement')
plt.ylim(0,5)
plt.xlim(0,50)
plt.legend(loc='upper left')
plt.title(f"Double-Difference Residuals \n Date range {start_date}-{end_date} \n GNSS simple subtraction displacement")
plt.xlabel("Distance (km)")
plt.ylabel("Amplitude of Double-Differenced Displacement Residual (cm)")
plt.show()

out_fig = os.path.abspath('coseismic_insar-gnss_vs_distance.png')
fig.savefig(out_fig, bbox_inches='tight', transparent=True, dpi=300)

<a id='nisar_anal'></a>
## 4.4. GNSS-InSAR Residuals Analysis with Step Function


In [ ]:
# version of GNSS coseismic displacements from full time-series fit
#Create dictionary with the stations as the key and all their info as an array 
stn_fit_dict = {}  

#Loop over GNSS station locations
for i in range(0,len(site_names)): 
    
    # convert GNSS station lat/lon information to InSAR x/y grid
    # convert GNSS station lat/lon information to InSAR x/y grid
    stn_lat = site_lats[i]
    stn_lon = site_lons[i]
    x_value = round((stn_lon - W)/lon_step)
    y_value = round((stn_lat - N)/lat_step)
    
    # get velocities and residuals
    disp_GNSS_fit = gnss_los_coseis_cm[i]
    #Caution: If you expand the radius parameter farther than the bounding grid it will break. 
    #To fix, remove the station in section 4 when the site_names list is filtered
    disp_px_rad = EQstep[y_value-pixel_radius:y_value+1+pixel_radius, 
                     x_value-pixel_radius:x_value+1+pixel_radius]
    disp_InSAR = np.median(disp_px_rad)

    residual_fit = disp_GNSS_fit - disp_InSAR

    # populate data structure
    values = [x_value, y_value, disp_InSAR, disp_GNSS_fit, residual, stn_lat, stn_lon]
    stn = site_names[i]
    stn_fit_dict[stn] = values
    
# extract data from structure
res_list2 = []
insar_disp2 = []
gnss_disp2 = []
lat_list2 = []
lon_list2 = []
for stn in site_names: 
    insar_disp2.append(stn_fit_dict[stn][2])
    gnss_disp2.append(stn_fit_dict[stn][3])
    res_list2.append(stn_fit_dict[stn][4])
    lat_list2.append(stn_fit_dict[stn][5])
    lon_list2.append(stn_fit_dict[stn][6])
num_stn = len(site_names) 
print('Finish creating InSAR residuals at GNSS sites')

<a id='make_velres2'></a>
## 4.5. Make Double-differenced Displacement Residuals Step


In [ ]:
stn_dict = stn_fit_dict # use the fit displacements for the rest of the calculations

diff_res_fit_list2 = []
stn_dist_list2 = []
dict_keys = list(stn_dict.keys())

# remove reference stn
site_names_analysis = list(site_names)
#site_names_analysis.remove(gps_ref_site_name)

# loop over stations
for i in range(len(site_names_analysis)-1):
    stn1 = dict_keys[i]
    for l in range(i + 1, len(dict_keys)):
        stn2 = dict_keys[l]
        # calculate between-station velocity residual
        # stn_dict values = [x_value, y_value, vel_InSAR, vel_GPS, residual, stn_lat, stn_lon]
        # index 3 = gps vel
        gps_vel_diff = stn_dict[stn1][3] - stn_dict[stn2][3]
        # index 2 = insar vel 
        insar_vel_diff = stn_dict[stn1][2]-stn_dict[stn2][2]
        # calculate double-difference
        diff_res = gps_vel_diff - insar_vel_diff
        diff_res_fit_list2.append(diff_res)
        # get distance between selected station
        # index 5 is lat, 6 is lon
        dlat = (stn_dict[stn1][5]-stn_dict[stn2][5])
        dlon = (stn_dict[stn1][6]-stn_dict[stn2][6])*np.sin(stn_dict[stn1][5])
        #convert degrees to km
        stn_dist = math.sqrt(dlat**2 + dlon**2)*111
        stn_dist_list2.append(stn_dist)

<a id='ampvsdist2'></a>
## 4.6. Amplitude vs. Distance of Double-differences Step (not quite a structure function)


In [ ]:
abs_ddiff_disp_fit = [abs(i) for i in diff_res_fit_list2]

# Write data for statistical tests
dist = [np.array(stn_dist_list2)]  # in km
rel_measure = [np.array(np.abs(diff_res_fit_list2))*10.0]  # in mm
ifgs_date = np.array([[dt.strptime(start_date,"%Y%m%d"),dt.strptime(end_date,"%Y%m%d")]])
#n_ifgs = len(dist)
n_ifgs = 1

fig, ax = plt.subplots(figsize=(11,7))
plt.scatter(stn_dist_list2,abs_ddiff_disp_fit,label='D_gnss_fit - D_InSAR for station pair')
plt.plot(dist_th, acpt_error_cm, 'r',label='Coseismic requirement')
plt.ylim(0,5)
plt.xlim(0,50)
plt.legend(loc='upper left')
plt.title(f"Double-Difference Residuals \n Date range {start_date}-{end_date} \n GNSS time-series fit displacement")
plt.xlabel("Distance (km)")
plt.ylabel("Amplitude of Double-Differenced Displacement Residual (cm)")
plt.show()

out_fig = os.path.abspath('coseismic_insar-gnss_fit_vs_distance.png')
fig.savefig(out_fig, bbox_inches='tight', transparent=True, dpi=300)



<div class="alert alert-warning">
Final result Method 1—Succesful when 68% of points below requirements line
</div>


<a id='coseismic_valid_method1'></a>
## 4.7. Coseismic Requirement Validation: Method 1


Count number of double-difference residuals from the step-function time-series fit above and below the coseismic requirements curve and calculate percentage of total for a series of distance bins. Compare to 68.3% threshold in each bin and for all points.

In [ ]:
# Calculate Statistics
n_bins = 10
threshold = 0.683  
#  we assume that the distribution of residuals is Gaussian and 
#  that the threshold represents a 1-sigma limit within which 
#  we expect 68.3% of residuals to lie.

bins = np.linspace(0.1, 50.0, num=n_bins+1)
n_all = np.empty([n_ifgs, n_bins+1], dtype=int) # number of points for each ifgs and bins
n_pass = np.empty([n_ifgs, n_bins+1], dtype=int) # number of points pass

# the final column is the ratio as a whole
for i in range(n_ifgs):
    inds = np.digitize(dist[i], bins)
    for j in range(1, n_bins + 1):
        rqmt = 4*(1+np.sqrt(dist[i][inds==j]))# mission requirement for i-th ifgs and j-th bins in mm
        rem = rel_measure[i][inds == j] # relative measurement
        #assert len(rqmt) == len(rem)
        n_all[i,j-1] = len(rem)
        n_pass[i,j-1] = np.count_nonzero(rem < rqmt)
    n_all[i,-1] = np.sum(n_all[i, 0:-2])
    n_pass[i,-1] = np.sum(n_pass[i, 0:-2])
ratio = n_pass / n_all
success_or_fail = ratio > threshold

def to_str(x:bool):
    if x==True:
        return 'true '
    elif x==False:
        return 'false '
success_or_fail_str = [list(map(to_str, x)) for x in success_or_fail]

columns = []
for i in range(n_bins):
    columns.append(f'{bins[i]:.2f}-{bins[i+1]:.2f}')
columns.append('total')

index = []
for i in range(len(ifgs_date)):
    index.append(ifgs_date[i,0].strftime('%Y%m%d')+'-'+ifgs_date[i,1].strftime('%Y%m%d'))
    
# Display Results

n_all_pd = pd.DataFrame(n_all,columns=columns,index=index)
n_pass_pd = pd.DataFrame(n_pass,columns=columns,index=index)
ratio_pd = pd.DataFrame(ratio,columns=columns,index=index)
success_or_fail_pd = pd.DataFrame(success_or_fail_str,columns=columns,index=index)

display(n_all_pd)  # Number of data points in each bin
display(n_pass_pd) # Number of data points that lie below the curve

#Set new style for table
s = ratio_pd.style
s.set_table_styles([  # create internal CSS classes
    {'selector': '.true', 'props': 'background-color: #e6ffe6;'},
    {'selector': '.false', 'props': 'background-color: #ffe6e6;'},
], overwrite=False)
display(s.set_td_classes(success_or_fail_pd))  # Percentage of passing points:
display(success_or_fail_pd)  # Explicit pass/fail table

# Overall pass/fail criterion
if n_ifgs == 1:
    if success_or_fail_pd.iloc[0]['total']:
        print("This coseismic displacement dataset passes the requirement.")
    else:
        print("This coseismic displacement dataset does not pass the requirement.")

<div class="alert alert-warning">
Final result Method 1 table by distance bin—successful when greater than 0.683
</div>


<a id='nisar_validation2'></a>
# 5. NISAR Validation Approach 2: InSAR-only Structure Function

In Validation approach 2, we use a date when there was no earthquake and do the same step function fit.

In [ ]:
os.chdir(mintpy_dir)  # reset directory in case running out of sequence
print(mintpy_dir)
# Step function fit when there was no earthquake
command = 'timeseries2velocity.py ' + timeseries_filename + ' --step ' + sitedata['sites'][site]['noEarthquakeDate']
process = subprocess.run(command, shell=True)


Now we can view the step function amplitude for the non-earthquake date and the new linear velocity estimate. Because we have the step function at a time far away from the Ridgecrest earthquake, the step function has a small amplitude and the linear velocity has absorbed the earthquake displacements. The non-earthquake step plot has same color scale as earthquake coseismic plot above to show the small atmospheric noise of a date without an earthquake.

In [ ]:
scp_args = 'velocity.h5 step' + sitedata['sites'][site]['noEarthquakeDate'] + ' -v -40 40 --ref-yx 200 950 --colormap RdBu --figtitle LOS_Coseismic' # --plot-setting ' + plot_config_file
view.main(scp_args.split())
scp_args = 'velocity.h5 velocity -v -25 25 --ref-yx 200 950 --colormap RdBu --figtitle LOS_Velocity' # --plot-setting ' + plot_config_file
view.main(scp_args.split())

<a id='array_mask'></a>
## 5.1. Read Array and Mask Pixels with no Data

In [ ]:
# use the non-earthquake displacement as the insar_displacment for statistics and convert to mm
noEQdataset = 'step' + sitedata['sites'][site]['noEarthquakeDate']
noEQstep,atrib = readfile.read(vel_file, datasetName = noEQdataset)  #read and coseismic step 
noEQdate = sitedata['sites'][site]['noEarthquakeDate']

insar_displacement = noEQstep[np.newaxis, :] * 1000. # convert from m to mm and add dimension to array to allow multiple datasets

ifgs_date = np.array([noEQdate])  # only one non-earthquake date for now
#print(insar_displacement.shape)
n_ifgs = insar_displacement.shape[0]

# mask out no-data areas
insar_displacement[insar_displacement == 0] = np.nan

# display map of data after masking
cmap = plt.get_cmap('RdBu')
for i in range(n_ifgs):
    fig, ax = plt.subplots(figsize=[18, 5.5])
    img1 = ax.imshow(insar_displacement[i], cmap=cmap, interpolation='nearest', extent=(W, E, S, N))
    ax.set_title("Coseismic \n Date "+noEQdate)
    cbar1 = fig.colorbar(img1, ax=ax)
    cbar1.set_label('LOS displacement [mm]')

<a id='remove_trend'></a>
## 5.2. Randomly Sample Pixels and Pair Them Up with Option to Remove Trend

First, calculate the coordinate for every pixel.

Then for each non-earthquake step fit, randomly selected pixels need to be paired up. In order to keep measurements independent, different pixel pairs can not share same pixel. This is achieved by pairing up in sequence, i.e., pairing up pixel number 1 and number 2, 3 and 4...

In [ ]:
X0,Y0 = load_geo(atrib)
X0_2d,Y0_2d = np.meshgrid(X0,Y0)

M2dist = []; rel_measure = []
for i in range(n_ifgs):
    dist_i, rel_measure_i = samp_pair(X0_2d,Y0_2d,insar_displacement[i],num_samples=1000000)
    M2dist.append(dist_i)
    rel_measure.append(rel_measure_i)

Check the statistical property of selected pixel pairs and overall histogram of relative measurements

In [ ]:
for i in range(n_ifgs):
    fig, ax = plt.subplots(figsize=[18, 5.5])
    img1 = ax.hist(M2dist[i], bins=100)
    ax.set_title(f"Histogram of distance \n Coseismic Date "+noEQdate)
    ax.set_xlabel(r'Distance ($km$)')
    ax.set_ylabel('Frequency')
    ax.set_xlim(0,50)
    
for i in range(n_ifgs):
    fig, ax = plt.subplots(figsize=[18, 5.5])
    img1 = ax.hist(rel_measure[i], bins=100)
    ax.set_title(f"Histogram of Relative Measurement \n Coseismic Date "+noEQdate)
    ax.set_xlabel(r'Relative Measurement ($mm$)')
    ax.set_ylabel('Frequency')

<a id='M2ampvsdist2'></a>
## 5.3. Amplitude vs. Distance of Relative Measurements (pair differences)


In [ ]:
dist_th = np.linspace(0,50,100)  # distance in km
rqmt = 4*(1+np.sqrt(dist_th))   # coseismic requirement in mm
for i in range(n_ifgs):
    fig, ax = plt.subplots(figsize=[18, 5.5])
    ax.plot(dist_th, rqmt, 'r',label='Coseismic requirement')
    ax.scatter(M2dist[i], rel_measure[i], s=1, alpha=0.25, label='Relative displacement for pixel pair')
    ax.set_title(f"Method 2: Relative Measurements between Pixel Pairs and Requirement Curve vs. Distance \n"+site+" Non-earthquake Date "+noEQdate)
    ax.set_ylabel(r'Relative Measurement ($mm$)')
    ax.set_xlabel('Distance (km)')
    ax.set_xlim(0,50)
    plt.legend(loc='upper left')

    
    plt.show()

    out_fig = os.path.abspath('coseismic_insar-only_vs_distance_'+site+'_date'+noEQdate+'.png')
    fig.savefig(out_fig, bbox_inches='tight', transparent=True, dpi=300)
    

<div class="alert alert-warning">
Final result Method 2—
    68% of points below the requirements line is success
</div>


Save data used of approach 2:
- `dist`: distance of pixel pairs,
- `rel_measure`: relative measurement of pixel pairs,
- `ifgs_date`: list of dates for coseismic step fits

In [ ]:
with open(os.path.join(work_dir,'approach2.pkl'),'wb') as f:
    pickle.dump((M2dist,rel_measure,ifgs_date),f)

<a id='M2RelMeasTable'></a>
## 5.4. Bin Sample Pairs by Distance Bin and Calculate Statistics

In approach 2, the number of pixel pairs which meet the mission requirement as a percentage of the total number of pixel pairs selected are counted.

The method we apply to evaluate the noise structure is similar to that in the InSAR-GNSS comparison. We count the percentage of measurements that fall below the threshold curve  for each of the 5-km-wide bins. If the average of the percentages from all bins is larger than 0.683, we judge that the noise level falls below the requirement.

Then we prepare table of results.

In [ ]:
n_bins = 10
bins = np.linspace(0.1,50.0,num=n_bins+1)

n_all = np.empty([n_ifgs,n_bins+1],dtype=int) # number of points for each ifgs and bins
n_pass = np.empty([n_ifgs,n_bins+1],dtype=int) # number of points pass
#ratio = np.empty([n_ifgs,n_bins+1]) # ratio
# the final column is the ratio as a whole
for i in range(n_ifgs):
    inds = np.digitize(M2dist[i],bins)
    for j in range(1,n_bins+1):
        rqmt = 4*(1+np.sqrt(M2dist[i][inds==j]))# coseismic mission requirement for i-th ifgs and j-th bins
        rem = rel_measure[i][inds==j] # relative measurement
        assert len(rqmt) == len(rem)
        n_all[i,j-1] = len(rem)
        n_pass[i,j-1] = np.count_nonzero(rem<rqmt)
    n_all[i,-1] = np.sum(n_all[i,0:-2])
    n_pass[i,-1] = np.sum(n_pass[i,0:-2])
    
ratio = n_pass/n_all
mean_ratio = np.array([np.mean(ratio[:,:-1],axis=1)])
ratio = np.hstack((ratio,mean_ratio.T))
thresthod = 0.683
#The assumed nature of Gaussian distribution gives a probability of 0.683 of being within one standard deviation.
success_or_fail = ratio>thresthod
success_or_fail_str = [list(map(to_str, x)) for x in success_or_fail]

# make table

columns = []
for i in range(n_bins):
    columns.append(f'{bins[i]:.2f}-{bins[i+1]:.2f}')
columns.append('total')

index = []
for i in range(len(ifgs_date)):
    index.append(ifgs_date[i])
    
n_all_pd = pd.DataFrame(n_all,columns=columns,index=index)
n_pass_pd = pd.DataFrame(n_pass,columns=columns,index=index)
ratio_pd = pd.DataFrame(ratio,columns=columns+['mean'],index=index)
success_or_fail_pd = pd.DataFrame(success_or_fail_str,columns=columns+['mean'],index=index)

Number of data points (pixel pairs) in each bin:

In [ ]:
n_all_pd

Number of data points that below the requirements curve:

In [ ]:
n_pass_pd

Ratio of pass to total samples:

In [ ]:
s = ratio_pd.style
s.set_table_styles([  # create internal CSS classes
    {'selector': '.true', 'props': 'background-color: #e6ffe6;'},
    {'selector': '.false', 'props': 'background-color: #ffe6e6;'},
], overwrite=False)
s.set_td_classes(success_or_fail_pd)

<div class="alert alert-warning">
Final result Method 2 table of distance bins—
    68% of points below the requirements line is success
</div>


Percentage of dates and bins that pass the requirement (0.683):

In [ ]:
np.count_nonzero(ratio_pd['total']>thresthod)/n_ifgs

<a id='appendix'></a>
# Appendix: Supplementary Comparisons and Plots


<a id='compare_raw'></a>
## A.1. Compare Raw Velocities


In [ ]:
vmin, vmax = -40, 40
plt.figure(figsize=(11,7))
plt.hist(insar_disp1, range = [vmin, vmax],bins = 40, color = "green",edgecolor='grey',label='D_InSAR')
plt.hist(gnss_disp1, range = [vmin, vmax],bins = 40, color = "orange",edgecolor='grey',label='D_gnss', alpha=0.5)
plt.legend(loc='upper right')
plt.title(f"Displacements \n Date range {EQpre_date}-{EQpost_date} \n Reference stn: {sitedata['sites'][site]['gps_ref_site_name']} \n Number of stations used: {num_stn}")
plt.xlabel('LOS displacement (cm)')
plt.ylabel('N Stations')
plt.ylim(0,20)
plt.show()

<a id='plot_vel'></a>
## A.2. Plot Velocity Residuals


In [ ]:
plt.figure(figsize=(11,7))
plt.hist(res_list1, bins = 40, range = [vmin, vmax],edgecolor='grey',color="darkblue",linewidth=1,label='D_gnss - D_InSAR (area average)')
plt.legend(loc='upper right')
plt.title(f"Residuals \n Date range {start_date}-{end_date} \n Reference stn: {sitedata['sites'][site]['gps_ref_site_name']} \n Number of stations used: {num_stn}")
plt.xlabel('Displacement Residual (cm)')
plt.ylabel('N Stations')
plt.ylim(0,15)
plt.show()

<a id='plot_velres'></a>
## A.3. Plot Double-differenced Residuals


In [ ]:
plt.figure(figsize=(11,7))
plt.hist(diff_res_list1, range = [vmin, vmax],bins = 40, color = "darkblue",edgecolor='grey',label='D_gnss_(s1-s2) - D_InSAR_(s1-s2)')
plt.legend(loc='upper right')
plt.title(f"Difference Residuals \n Date range {start_date}-{end_date} \n Reference stn: {sitedata['sites'][site]['gps_ref_site_name']} \n Number of stations used: {num_stn}")
plt.xlabel('Double Differenced Displacement Residual (cm)')
plt.ylabel('N Stations')
plt.ylim(0,250)
plt.show()

<a id='appendix_gps'></a>
## A.4. GPS Position Plots


In [ ]:
#Read in timeseries file
time_file = os.path.join(work_dir, 'MintPy/timeseries.h5')
insar_displacements = readfile.read(time_file, datasetName='timeseries')[0] * 100.

#Get aquisition dates, trim the str, and convert to datetime
raw_aqu_list = readfile.get_slice_list(time_file)
acquisitions_dates = []
for i in range(len(raw_aqu_list)):
    date = raw_aqu_list[i].split("-")
    #aqu_dates.append = date[1]
    acquisitions_dates.append(dt.strptime(date[1], "%Y%m%d"))
ndates = len(acquisitions_dates)

#Plot displacements and velocity timeseries at GNSS station locations
test_list = site_names
#test_list = ['P595'] #', 'CAHA', 'CAKC', 'CAND', 'CARH'] #for testing, remove after
for stn in test_list:
    
    #InSAR Info
    insar_timeseries = []
    stn_x = (stn_dict[stn][0])
    stn_y = (stn_dict[stn][1])
    for i in range(ndates):
        insar_displacement = insar_displacements[i,stn_y,stn_x]
        insar_timeseries.append(insar_displacement)     
    InSAR_stn_disp = stn_dict[stn][2]
    GNSS_stn_disp = stn_dict[stn][3]
    #print(InSAR_stn_vel)
    #print(GNSS_stn_vel)
    
    #get the length of time between first and last interferogram for velocity plotting
    insar_timespan = acquisitions_dates[ndates - 1] - acquisitions_dates[0]
    conversion = insar_timespan.days/365.25
    aqu_dates_convert=acquisitions_dates #/conversion
    
    #Plot InSAR
    plt.figure(figsize=(15,5))  
    plt.plot([0,100000],[0,0], color='grey',linestyle='dashed',linewidth=1)
    plt.scatter(acquisitions_dates, insar_timeseries, label="InSAR Position Time Series")
    #velocity trendline
    plt.plot([acquisitions_dates[0], acquisitions_dates[ndates - 1]],[0,InSAR_stn_disp],label=f"InSAR Displacement {InSAR_stn_disp:.3f} (Ref: {sitedata['sites'][site]['gps_ref_site_name']})")

    #GNSS Info
    gnss_obj = gnss.get_gnss_class('UNR')(site = stn, data_dir = os.path.join(mintpy_dir,'GNSS'))
    gnss_obj.open()
    dates = gnss_obj.dates
    # date range for this station
    statStart = gnss_obj.date_list[0]
    statEnd = gnss_obj.date_list[-1]

    dates, disp_los, std, site_lalo, ref_site_lalo = gnss_obj.get_los_displacement(geom_file, start_date=statStart, end_date=statEnd, ref_site=None,
                                  gnss_comp=gnss_comp, print_msg=False)
    
    #Plot GNSS
    index_begin = np.min(np.where(dates >= start_date_gnss))
    index_end = np.max(np.where(dates <= end_date_gnss))
    dates_cut = dates[index_begin:index_end]
    disp_los_cut = disp_los[index_begin:index_end]
    disp_los_cut = (disp_los_cut - np.median(disp_los_cut))*100
    plt.scatter(dates_cut, disp_los_cut - disp_los_cut[0], label="GNSS Daily Positions")   
    #velocity trendline
    plt.plot([acquisitions_dates[0], acquisitions_dates[ndates - 1]],[0,GNSS_stn_disp],color='orange',label=f"GNSS Displacement {GNSS_stn_disp:.3f} (Ref: {sitedata['sites'][site]['gps_ref_site_name']})")

    plt.title(f"Station Name: {stn}") 
    plt.ylabel('Displacement (cm)')
    plt.ylim(-37,37) # plot range for Ridgecrest
    plt.xlim(aqu_dates_convert[0],aqu_dates_convert[ndates - 1])
    plt.legend(loc="best")
    